<a href="https://colab.research.google.com/github/natmartem/natmartem.github.io/blob/main/anomaly_detection_blank_ver3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Context
**In this notebook, you will implement various anomaly detection techniques** using a pretrained ResNet model and datasets like the [CIFAR-10](https://paperswithcode.com/dataset/cifar-10), [CIFAR-100](https://paperswithcode.com/dataset/cifar-100), and [Street View House Numbers](https://paperswithcode.com/dataset/svhn).

## Note:
The code will run much faster with a GPU. To enable this, click on **Runtime > Change runtime type > Hardware Accelerator > select GPU**.

It could be that a GPU is not is available. In this case, select "None" as a hardware accelerator. The code will still work.

## Contributers:
Jason Ding



--------

## Downloads

The following cells **install libaries and download a pretrained model**.

(Note: the downloads may take up to a minute to finish. You may get errors about dependency conflicts with `torchtext`, `torchdata`, and `torchaudio`. These libraries are not necessary so you can safely ignore these errors).

In [1]:
!wget https://github.com/hendrycks/outlier-exposure/raw/master/CIFAR/snapshots/baseline/cifar10_wrn_baseline_epoch_99.pt
!wget https://raw.githubusercontent.com/hendrycks/pre-training/master/robustness/adversarial/models/wrn_with_pen.py
!pip3 install torchvision==0.12.0

--2023-08-06 11:15:15--  https://github.com/hendrycks/outlier-exposure/raw/master/CIFAR/snapshots/baseline/cifar10_wrn_baseline_epoch_99.pt
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hendrycks/outlier-exposure/master/CIFAR/snapshots/baseline/cifar10_wrn_baseline_epoch_99.pt [following]
--2023-08-06 11:15:15--  https://raw.githubusercontent.com/hendrycks/outlier-exposure/master/CIFAR/snapshots/baseline/cifar10_wrn_baseline_epoch_99.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9037421 (8.6M) [application/octet-stream]
Saving to: ‘cifar10_wrn_baseline_epoch_99.pt’

cifar10_wrn_baselin 10

In [2]:
import math
import torch
import torchvision
import numpy as np

import sklearn.metrics as sk                    # https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
import torch.nn.functional as F                 # https://pytorch.org/docs/stable/nn.functional.html
from torchvision import datasets                # https://pytorch.org/vision/stable/datasets.html

from wrn_with_pen import WideResNet
from torch.utils.data import Dataset            # https://pytorch.org/docs/stable/data.html
import torchvision.transforms as transforms     # https://pytorch.org/vision/stable/transforms.html

prefetch = 2

## Load Model and Datasets

The **following cells load the ResNet model, CIFAR-10 dataset, and out-of-distribution dataset**.

In [3]:
# Load ResNet model

net = WideResNet(depth=40, num_classes=10, widen_factor=2, dropRate=0.3)

if torch.cuda.is_available():
    net.load_state_dict(torch.load('cifar10_wrn_baseline_epoch_99.pt'))
    net.eval()
    net.cuda()
else:
    net.load_state_dict(
        torch.load('cifar10_wrn_baseline_epoch_99.pt', map_location=torch.device('cpu'))
    )
    net.eval()

In [4]:
# ================================
# ====== Loading Datasets ========
# ================================

mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]



# CIFAR-10
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

cifar_10_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=data_transform
)

cifar10_data = cifar_10_data
cifar10_loader = torch.utils.data.DataLoader(
    cifar10_data,
    batch_size=200,
    shuffle=False,
    num_workers=prefetch,
    pin_memory=True                           # https://discuss.pytorch.org/t/when-to-set-pin-memory-to-true/19723
)

ood_num_examples = len(cifar10_data) // 5



# CIFAR-100
cifar100_ood_data = datasets.CIFAR100(
    root="data",
    train=False,
    download=True,
    transform=data_transform
)

cifar100_ood_loader = torch.utils.data.DataLoader(
    cifar100_ood_data,
    batch_size=200,
    shuffle=True,
    num_workers=prefetch,
    pin_memory=True
)



# Rademacher Noise
dummy_targets = torch.ones(ood_num_examples)
image_size = (ood_num_examples, 3, 32, 32)

ood_data = torch.from_numpy(
    np.random.binomial(n=1, p=0.5, size=image_size).astype(np.float32)
)
ood_data = ood_data * 2 - 1

rademacher_ood_data = torch.utils.data.TensorDataset(ood_data, dummy_targets)
rademacher_ood_loader = torch.utils.data.DataLoader(
    rademacher_ood_data,
    batch_size=200,
    shuffle=True
)



# Street View House Numbers
data_transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

svhn_ood_data = torchvision.datasets.SVHN(
    root = "data",
    split="test",
    transform = data_transform,
    download = True
)

svhn_ood_loader = torch.utils.data.DataLoader(
    svhn_ood_data,
    batch_size=200,
    shuffle=True,
    num_workers=prefetch,
    pin_memory=True
)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data


  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting data/cifar-100-python.tar.gz to data


  0%|          | 0/64275384 [00:00<?, ?it/s]

## Load Predefined Functions

These functions will test your implementations of anomaly score calculators. You don't need to do anything here.

In [5]:
concat = lambda x: np.concatenate(x, axis=0)
to_np = lambda x: x.data.cpu().numpy()

def get_ood_scores(
    loader : torch.utils.data.DataLoader,
    anomaly_score_calculator,
    model_net : torch.nn.Module,
    use_penultimate : bool=False) -> np.ndarray:

    '''
    Calculates the anomaly scores for a portion of the given dataset.

    Parameters
    --------------------
    loader (type: torch.utils.data.DataLoader)
    - Contains the batched data of a dataset.
    anomaly_score_calculator (type: function)
    - Finds anomaly score using last / 2nd last layer.
    model_net (type: torch.nn.Module)
    - The image classifier.
    use_penultimate (type: bool)
    - Indicates whether to use penultimate (2nd last) output layer.

    Returns
    ---------------------
    scores (type: numpy.ndarray, dim: (num_scores,), dtype: float)
    - Array of anomaly scores per batch of input.
    '''

    _score = []

    with torch.no_grad():
        # If no GPU, runs on smaller fraction of data
        if torch.cuda.is_available():
            fraction = 200
        else:
            fraction = 1000

        for batch_idx, (data, target) in enumerate(loader):
            if batch_idx >= (ood_num_examples // fraction):
                break

            if torch.cuda.is_available():
                data = data.cuda()

            # returns tuple (last layer activations, 2nd last layer activations)
            output = model_net(data)

            if use_penultimate:
                score = anomaly_score_calculator(output[0], output[1])
            else:
                score = anomaly_score_calculator(output[0])
            _score.append(score)

    return concat(_score).copy()

In [6]:
# ======================================================
# =================== Printing Utils ===================
# ======================================================


def get_and_print_results(
    ood_loader : torch.utils.data.DataLoader,
    anomaly_score_calculator,
    model_net : torch.nn.Module,
    use_penultimate : bool) -> float:

    '''
    Returns and prints out the AUROC score of a dataset.

    Parameters
    --------------------
    ood_loader (type: torch.utils.data.DataLoader)
    - Contains the batched data of a dataset.
    anomaly_score_calculator (type: function)
    - Finds anomaly score given last / 2nd last layer's activations.
    model_net (type: torch.nn.Module)
    - The image classifier.
    use_penultimate (type: bool)
    - Whether to use the penultimate (2nd last layer).

    Returns
    ---------------------
    auroc (type: float)
    - The auroc score calculated by the functions you implement.
    '''

    out_score = get_ood_scores(ood_loader, anomaly_score_calculator, model_net,
                               use_penultimate = use_penultimate)
    auroc = get_auroc(out_score, in_score) # in_score is global variable
    print('AUROC: \t\t\t{:.2f}'.format(100 * auroc) + "%")
    return auroc



all_anomaly_results = {}


def print_all_results(
    anomaly_score_calculator,
    anomaly_score_name : str,
    model_net : torch.nn.Module,
    model_name : str = "default_model",
    use_penultimate : bool = False) -> None:

    '''
    Prints out the AUROC score of all the OOD datasets.

    Warning: results added to global dict all_anomaly_results to display later.

    Parameters
    ---------------------
    anomaly_score_calculator (type: function)
    - Finds anomaly score given last / 2nd last layer's activations.
    anomaly_score_name (type: str)
    - The name of the anomaly score method. (To record results)
    model_net (type: torch.nn.Module)
    - The image classifier.
    model_name (type: str)
    - The name of the classifier model. (To record results)
    use_penultimate (type: bool)
    - Whether to use the penultimate (2nd last layer).

    Returns
    ---------------------
    None
    '''

    # Prep variables
    global in_score, all_anomaly_results # access/modify global var instances
    in_score = get_ood_scores(cifar10_loader, anomaly_score_calculator,
                              model_net, use_penultimate)
    results = []

    # Evaluate results
    labels = ['Rademacher Noise Detection', '\nSVHN Detection', '\nCIFAR-100 Detection']
    loaders = [rademacher_ood_loader, svhn_ood_loader, cifar100_ood_loader]

    for label, loader in zip(labels, loaders):
      print(label)
      auroc = get_and_print_results(loader, anomaly_score_calculator, model_net, use_penultimate)
      results.append(auroc)

    average = sum(results) / len(results)
    results.append(average)

    # Save results
    if not model_name in all_anomaly_results:
        all_anomaly_results[model_name] = {}
    all_anomaly_results[model_name][anomaly_score_name] = results

## Implement AUROC Score
Fill in the `get_auroc` function. This function will calculate the AUROC score of an out-of-distribution dataset.

Hint: you may find the [`sklearn.metrics.roc_auc_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html) function helpful. Both `_ood` and `_norm` should be used.

In [39]:
from sklearn.metrics import roc_auc_score
def get_auroc(_ood, _norm):
    '''
    Calculates the AUROC score of a OOD dataset.

    Parameters
    -------------------
    _ood (type: numpy.ndarray, dim: (num_scores,), dtype: float):
    - Anomaly scores of images in the out-of-distribution dataset
    _norm (type: numpy.ndarray, dim: (num_scores,), dtype: float)
    - Anomaly scores of images in the normal (CIFAR-10) dataset

    Returns
    -------------------
    float
    - The AUROC score the data in decimal form
    '''

    ############################################################################
    # TODO:  Calculate the AUROC score.                                        #
    ############################################################################
    auroc_score = 0.0
    num_scores = len(_ood)
    y_true = np.concatenate([np.ones(shape = (num_scores,)), np.zeros(shape = (num_scores,))])
    y_score = np.concatenate([_ood, _norm])
    auroc_score = roc_auc_score(y_true, y_score)
    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    return auroc_score

## Implement Anomaly Score Calculators
Fill in the folllowing functions which calculate the anomaly score given the model's output for a batch of data (the **model's output will contain one [logit](https://stackoverflow.com/questions/34240703/) per image in the batch**).

&nbsp;

The following equations show how the logits can be used to compute the anomaly score.

Max Logit Equation:

$ \LARGE \text{Score}=-\text{max}\ l_k$
- Where $k=1,2,...,\text{num_classes}$ and $l_k$ represents the logits for all $k$ classes.

&nbsp;

Max Softmax Equation:

$ \LARGE \text{Score}=-\text{max}\ p(y=k|x)$
- Where $p(y=k|x)$ represents the model's predicted probability that input $x$ has its training label $y$ equal to the $kth$ class (for all $k$ classes)

&nbsp;

Cross Entropy Anomaly Equation:

$ \LARGE \text{Score} = \bar{l}-\text{log}∑_{k=1}^{\text{num_classes}}e^{l_k}$
- Where $\bar{l}$ represents the mean logit value and $k$ and $l_k$ are defined like above.

In [52]:
# ======================================================
# =================== Anomaly Scores ===================
# ======================================================



def max_logit_anomaly_score(output):
    '''
    Calculates the max logit anomaly score of a batch of outputs.

    Parameters
    ---------------------
    output (type: torch.Tensor, dim: (num_examples, num_classes), dtype: float)
    - The model's output logits for a batch of data.

    Returns
    ---------------------
    anomaly_score
    '''

    ############################################################################
    # TODO:  Calculate the max logit anomaly score.                            #
    ############################################################################
    score = None
    score = -torch.max(output, dim = 1).values

    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    return score.cpu()



def max_softmax_anomaly_score(output):
    '''
    Calculates the max softmax anomaly score of a batch of outputs.

    Parameters
    ---------------------
    output (type: torch.Tensor, dim: (num_examples, num_classes), dtype: float)
    - The model's output logits for a batch of data.

    Returns
    ---------------------
    anomaly_score
    '''

    ############################################################################
    # TODO:  Calculate the max softmax anomaly score.                          #
    ############################################################################
    score = None
    score = -torch.softmax(output, dim = 1).max(dim=1).values

    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    return score.cpu()



def cross_entropy_anomaly_score(output):
    '''
    Calculates the cross entropy anomaly score of a batch of outputs.

    Parameters
    ---------------------
    output (type: torch.Tensor, dim: (num_examples, num_classes), dtype: float)
    - The model's output logits for a batch of data.

    Returns
    ---------------------
    anomaly_score
    '''

    ############################################################################
    # TODO:  Calculate the cross entropy anomaly score.                        #
    ############################################################################
    score = None
    my_exponentas = torch.exp(output)
    my_sum = torch.sum(my_exponentas, dim=1)
    my_log = torch.log(my_sum)
    score = torch.mean(output, dim=1) - my_log

    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    return score.cpu()

## Print AUROC Results

Run the following cells in order to see how well each of the anomaly score calculators do on the OOD datasets.

In [49]:
print("======= Max Logit AUROC Scores =======")
print_all_results(max_logit_anomaly_score, "Max Logit", net)

======= Max Logit AUROC Scores =======
Rademacher Noise Detection
(2000,)
(2000,)
[1. 1. 1. ... 0. 0. 0.]
[-10.512507 -10.648192 -10.426215 ... -14.074633  -4.703778 -11.438648]
AUROC: 			70.66%

SVHN Detection
(2000,)
(2000,)
[1. 1. 1. ... 0. 0. 0.]
[ -5.2972665  -5.9091806 -10.434359  ... -14.074633   -4.703778
 -11.438648 ]
AUROC: 			90.88%

CIFAR-100 Detection
(2000,)
(2000,)
[1. 1. 1. ... 0. 0. 0.]
[ -7.855615  -10.230577   -3.2131584 ... -14.074633   -4.703778
 -11.438648 ]
AUROC: 			87.31%


In [53]:
print("======= Max Softmax AUROC Scores =======")
print_all_results(max_softmax_anomaly_score, "Max Softmax", net)

======= Max Softmax AUROC Scores =======
Rademacher Noise Detection
(2000,)
(2000,)
[1. 1. 1. ... 0. 0. 0.]
[-0.9682575  -0.50086933 -0.99474746 ... -0.9999912  -0.680868
 -0.999962  ]
AUROC: 			80.65%

SVHN Detection
(2000,)
(2000,)
[1. 1. 1. ... 0. 0. 0.]
[-0.74307483 -0.69456863 -0.78610444 ... -0.9999912  -0.680868
 -0.999962  ]
AUROC: 			92.15%

CIFAR-100 Detection
(2000,)
(2000,)
[1. 1. 1. ... 0. 0. 0.]
[-0.8030372 -0.9277797 -0.9381026 ... -0.9999912 -0.680868  -0.999962 ]
AUROC: 			88.19%


In [54]:
print("======= Cross Entropy AUROC Scores =======")
print_all_results(cross_entropy_anomaly_score, "Cross Entropy", net)

======= Cross Entropy AUROC Scores =======
Rademacher Noise Detection
(2000,)
(2000,)
[1. 1. 1. ... 0. 0. 0.]
[-10.97749  -11.16936   -9.182621 ... -14.074655  -5.088176 -11.438692]
AUROC: 			70.70%

SVHN Detection
(2000,)
(2000,)
[1. 1. 1. ... 0. 0. 0.]
[ -9.111313   -7.6525855 -14.443749  ... -14.074655   -5.088176
 -11.438692 ]
AUROC: 			90.43%

CIFAR-100 Detection
(2000,)
(2000,)
[1. 1. 1. ... 0. 0. 0.]
[ -5.993517  -7.873546 -14.066553 ... -14.074655  -5.088176 -11.438692]
AUROC: 			86.59%


## Implement ViM

You will now implement [virtual-logit matching (ViM)](https://arxiv.org/abs/2203.10807). It would be helpful to reread Section 4 of the paper, which describes the process of computing the principal space and alpha. This is a rough summary.

ViM calculates the **principal space** ($P^\perp$) using the model's penultimate (second last) layer activations on CIFAR-10 data. Technical notes:
- Use the 12 most significant principal components for your principal space.
- HINT: You may find it helpful to use [`np.linalg.svd`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html) and then principal component analysis. It is also possible to use covarience to find the principal space.

&nbsp;

Then, ViM calculates **alpha** ($\alpha$) by projecting the training data's features onto the principal space calculated before. This is known as the **residual** ($x^{P^\perp}$). Alpha is then calculated by the following equation:
> $ \LARGE \alpha := \frac{∑^k_{i=1}\text{max}_{j=1,...,C}\{l^i_j\}}{∑^K_{i=1}\| x_i^{P\perp} \|}$

In other words, alpha is the sum of each logit's maximum value, divided by the sum of the norms of each feature's residuals.

Your calculated alpha should be in the range of 1.5 to 1.9.


In [64]:
# You may find the following functions useful
from numpy.linalg import pinv, norm, eig              # https://numpy.org/doc/stable/reference/routines.linalg.html
from sklearn.covariance import EmpiricalCovariance    # https://scikit-learn.org/stable/modules/generated/sklearn.covariance.EmpiricalCovariance.html
from sklearn.decomposition import PCA

_score = []

# Extract fully connected layer's weights and biases
w = net.fc.weight.cpu().detach().numpy()
b = net.fc.bias.cpu().detach().numpy()

# Origin of a new coordinate system of feature space to remove bias
u = -np.matmul(pinv(w), b)


def compute_ViM_principal_space_and_alpha(
    training_data_loader : torch.utils.data.DataLoader,
    model_net : torch.nn.Module,
    verbose : bool=False) -> list:
    '''
    Calculates and returns the principal space and alpha values given
    the training data the model used.

    Parameters
    --------------------
    training_data_loader (type: torch.utils.data.DataLoader)
    - Contains the training dataset.
    model_net (type: torch.nn.Module)
    - The classifier model.
    verbose (type: bool)
    - If true, will print out the alpha value.

    Returns
    ---------------------
    principal_space (type: numpy.ndarray, dim: 128 x 128)
    - See equation above.
    alpha (type: float)
    - See equation above.
    '''

    # Get the first batch of training data to calculate principal space and alpha
    training_data, target = next(iter(training_data_loader))
    if torch.cuda.is_available():
        training_data = training_data.cuda()

    result = model_net(training_data)
    logit = result[0]                   # Logits (values before softmax)
    penultimate = result[1]             # Penultimate (values before fully connected layer)

    logit_id_train = logit.cpu().detach().numpy().squeeze()           # dim: 200 x 10
    feature_id_train = penultimate.cpu().detach().numpy().squeeze()   # dim: 200 x 128


    ############################################################################
    # TODO:  Calculate the pricipal space and then comput alpha.               #
    ############################################################################
    alpha = 0.0
    principal_space = None

    if verbose:
        print('Computing principal space...')

    #inner_principal_space, s, Vh = np.linalg.svd(penultimate)
    pca = PCA(12)
    residual = pca.fit_transform(feature_id_train)

    if verbose:
        print('Computing alpha...')

    # alpha is the sum of each logit's maximum value, divided by the sum of the norms of each feature's residuals.
    print('Your calculated alpha should be in the range of 1.5 to 1.9')
    residuals = pca.fit_transform(feature_id_train)
    sum_of_my_maximums = torch.max(result[0], dim=1).values.sum()
    sum_of_my_residual_norms = norm(residuals, axis=1).sum()
    alpha = sum_of_my_maximums/sum_of_my_residual_norms
    principal_space = residuals

    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################


    if verbose:
        print(f'alpha = {alpha}')

    return principal_space.T, alpha

principal_space, alpha = compute_ViM_principal_space_and_alpha(cifar10_loader, net, verbose = True)

Computing principal space...
Computing alpha...
Your calculated alpha should be in the range of 1.5 to 1.9
alpha = 2.2581145763397217


## Implement ViM Anomaly Score Calculator

Now, implement the ViM anomaly score calculator.

First, project the penultimate values onto the principal space from above, which is called the residual. Then, multiply the norm of this residual by alpha to get the **virtual logit score** ($\text{vlogit}$).

Next, compute the **energy score** ($\text{energy}$) by taking [LogSumExp](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.logsumexp.html) of the logits.

Finally, the outputted **anomaly score** is calculated by subtracting the virtual logit by the energy score.

$ \LARGE \text{vlogit}= \alpha \| {x^{P^\perp}} \|$\
$ \LARGE \text{energy} = \text{ln}\sum_{i=1}^C e^{l_i}$\
$ \LARGE \text{anomaly_score} = \text{vlogit} - \text{energy}$

In [ ]:
from scipy.special import logsumexp


def ViM_anomaly_score_calculator(output, penultimate):
    '''
    Calculates the ViM anomaly score of a batch of outputs.

    Parameters
    ----------------------
    output (type: torch.Tensor, dim: 200 x 10)
    - The model's output for a batch of data.
    penultimate (type: torch.Tensor, dim : 200 x 128)
    - The model's penultimate (second-last) layer values for a batch of data.

    Returns
    -----------------------
    score_id (type: numpy.ndarray: dim: 200)
    - The anomaly scores for a batch of data.
    '''

    logit_id_val = output.cpu().detach().numpy().squeeze()          # dim: 200 x 10
    feature_id_val = penultimate.cpu().detach().numpy().squeeze()   # dim: 200 x 128

    ############################################################################
    # TODO:  Calculate the anomaly score.                                      #
    ############################################################################
    score_id = None



    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################

    return score_id

print("======= ViM_anomaly_score_calculator =======")

# Make sure you have the correct ViM values before calculating the score
w, b = net.fc.weight.cpu().detach().numpy(), net.fc.bias.cpu().detach().numpy()
u = -np.matmul(pinv(w), b)
principal_space, alpha = compute_ViM_principal_space_and_alpha(cifar10_loader, net)

print_all_results(ViM_anomaly_score_calculator, "ViM", net, use_penultimate = True)

## Compare Anomaly Score Results
Run the following cell to see how the different anomaly score calculators compare to each other for the OOD datasets. You should see that ViM is superior to other anomaly scores in all of the datasets.

In [ ]:
# ======================================================
# ================== Compare Results ===================
# ======================================================

def get_results_max(model_name = "normal"):
    '''
    Computes the maximum anomaly score across a range of models and techniques
    '''

    all_anomaly_results[model_name]["max"] = [0,0,0,0,0]

    for key in all_anomaly_results[model_name].keys():
        if (key != "max"):
            index = 0

            for score in all_anomaly_results[model_name][key]:
                all_anomaly_results[model_name]["max"][index] = \
                    max(score, all_anomaly_results[model_name]["max"][index])
                index += 1


def compare_all_results():
    '''
    Creates a nice table to show anomaly scores for many datasets/techniques
    '''

    for model_name in all_anomaly_results:
        to_be_printed = " " * (25 - len(model_name)) + model_name
        dataset_names = ["Rademacher", "SVHN", "CIFAR-100", "Average"]
        for name in dataset_names:
            to_be_printed += " | " + " "*(6-math.ceil(len(name)/2)) + \
                                name + " "*(6-math.floor(len(name)/2))

        print(to_be_printed)
        print("=" * (25 + len(dataset_names) * 15))

        get_results_max(model_name = model_name)
        for key in all_anomaly_results[model_name].keys():
            if (key != "max"):
                to_be_printed = " "*(25-len(key)) + key
                index = 0

                for result in all_anomaly_results[model_name][key]:
                    if (all_anomaly_results[model_name]["max"][index] == result):
                        result = "*" + '{:.2f}'.format(round(result * 100, 2)) + "%"
                    else:
                        result = '{:.2f}'.format(round(result * 100, 2)) + "%"
                    to_be_printed += " | " + " "*(6-math.ceil(len(result)/2)) + \
                                        result + " "*(6-math.floor(len(result)/2))
                    index += 1

                print(to_be_printed)
        print()

    print("\n* highlights the maximum AUROC Score for an OOD Dataset")

compare_all_results()

## Data Augmentation

Now, check if models trained using data augmentation methods for robustness help in out-of-distribution detection.

Load a CIFAR-10 model that used PixMix data augmentation during training, and see how it fares compared to the default model that did not use data augmentation.

In [ ]:
!gdown 1mjIfbb3mfXXvAZ1sBnjotFr5yYFmLi68 # Downloading PixMix model
!gdown 1skZT6yplO-Sv4M8Ksgzx14cTY3H-HgOa # Downlaoding wideresnet class

In [ ]:
from wideresnet_with_pen import WideResNet as WideResNet2

In [ ]:
# Loading the PixMix model

pixmix_net = WideResNet2(depth=40, num_classes=10, widen_factor=4, drop_rate=0.3)
pixmix_net = torch.nn.DataParallel(pixmix_net)

if torch.cuda.is_available():
    checkpoint = torch.load('checkpoint.pth.tar')
    pixmix_net.load_state_dict(checkpoint['state_dict'])
    net.eval()
    net.cuda()
else:
    checkpoint = torch.load('checkpoint.pth.tar', map_location=torch.device('cpu'))
    pixmix_net.load_state_dict(checkpoint['state_dict'])
    net.eval()

## PixMix Model Testing

Now, test the PixMix model with the same anomaly score calculators you coded before.

In [ ]:
print("======= Max Logit AUROC Scores =======")
print_all_results(max_logit_anomaly_score, "Max Logit", pixmix_net, model_name = "pixmix_trained_model")

In [ ]:
print("======= Max Softmax Probability AUROC Scores =======")
print_all_results(max_softmax_anomaly_score, "Max Softmax Probability", pixmix_net, model_name = "pixmix_trained_model")

In [ ]:
print("======= Cross Entropy AUROC Scores =======")
print_all_results(cross_entropy_anomaly_score, "Cross Entropy", pixmix_net, model_name = "pixmix_trained_model")

In [ ]:
print("======= ViM_anomaly_score_calculator =======")
w, b = pixmix_net.module.fc.weight.cpu().detach().numpy(), pixmix_net.module.fc.bias.cpu().detach().numpy()
u = -np.matmul(pinv(w), b)
principal_space, alpha = compute_ViM_principal_space_and_alpha(cifar10_loader, pixmix_net)
print_all_results(ViM_anomaly_score_calculator, "ViM", pixmix_net, model_name = "pixmix_trained_model", use_penultimate = True)

## Compare No-data-augmentation Model vs PixMix Model

Let us now compare how the default model compared to the PixMix model by running the following cell.

You should see that the PixMix model successfully helps us in OOD detection, and has a higher AUROC score.

In [ ]:
compare_all_results()